# Lab 2: Machine Learning with PyTorch

**Duration:** 90-120 minutes | **Difficulty:** Intermediate

---

## Overview

This lab teaches PyTorch fundamentals through hands-on implementation of ML algorithms.

### Lab Structure

| Part | Topic | Key Concepts |
|------|-------|---------------|
| **Part 1** | PyTorch Tensors | Creating tensors, operations, autograd |
| **Part 2** | Linear Regression | Training loop, MSE loss, nn.Module |
| **Part 3** | Logistic Regression | Sigmoid, BCE loss, classification |
| **Part 4** | Support Vector Machines | Kernels, margins, support vectors |
| **Part 5** | Model Evaluation | Confusion matrix, precision, recall, F1 |

### Instructions

- Read each markdown cell carefully
- Write your code in the empty code cells
- Run cells with `Shift+Enter`

## Setup

Run the cell below to import the required libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.datasets import make_classification, make_moons

plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)
torch.manual_seed(42)

print("Setup complete!")
print(f"PyTorch version: {torch.__version__}")

---
# Part 1: PyTorch Tensors

Tensors are the fundamental data structure in PyTorch - similar to NumPy arrays but with GPU support and automatic differentiation.

## 1.1 Creating Tensors

Create tensors using these functions:

| Function | Description | Example |
|----------|-------------|----------|
| `torch.tensor([...])` | From Python list | `torch.tensor([1, 2, 3])` |
| `torch.zeros(shape)` | Zeros tensor | `torch.zeros(3, 4)` |
| `torch.ones(shape)` | Ones tensor | `torch.ones(2, 3)` |
| `torch.randn(shape)` | Random normal | `torch.randn(3, 3)` |
| `torch.from_numpy(arr)` | From NumPy | `torch.from_numpy(np_array)` |

**Your Task:** Create the following tensors:
1. `t1`: A 1D tensor with values [1.0, 2.0, 3.0, 4.0, 5.0]
2. `t2`: A 3×3 tensor of zeros
3. `t3`: A 2×4 tensor of random values
4. `t4`: Convert this NumPy array to a tensor: `np.array([[1, 2], [3, 4]])`

Print each tensor and its shape using `.shape`.

**Expected Output:**
```
t1: tensor([1., 2., 3., 4., 5.])
t1 shape: torch.Size([5])

t2:
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
t2 shape: torch.Size([3, 3])

t3 shape: torch.Size([2, 4])

t4:
tensor([[1, 2],
        [3, 4]])
t4 shape: torch.Size([2, 2])
```

**Sample Code:**
```python
# Creating tensors in different ways
my_tensor = torch.tensor([10.0, 20.0, 30.0])
my_ones = torch.ones(2, 2)
np_arr = np.array([5, 6, 7])
from_np = torch.from_numpy(np_arr)
print("Tensor:", my_tensor)
print("Shape:", my_tensor.shape)
```

In [ ]:
# Your code here


## 1.2 Tensor Operations

Common tensor operations:

| Operation | Description | Example |
|-----------|-------------|----------|
| `a + b` | Element-wise addition | `t1 + t2` |
| `a * b` | Element-wise multiplication | `t1 * t2` |
| `a @ b` or `torch.matmul(a, b)` | Matrix multiplication | `t1 @ t2` |
| `t.sum()` | Sum all elements | `tensor.sum()` |
| `t.mean()` | Mean of elements | `tensor.mean()` |
| `t.reshape(shape)` | Reshape tensor | `t.reshape(2, 3)` |

**Your Task:** 
1. Create two tensors: `a = torch.tensor([[1., 2.], [3., 4.]])` and `b = torch.tensor([[5., 6.], [7., 8.]])`
2. Calculate and print:
   - `add_result`: Element-wise addition of a and b
   - `mult_result`: Element-wise multiplication of a and b
   - `matmul_result`: Matrix multiplication of a and b
   - `sum_a`: Sum of all elements in a
   - `mean_b`: Mean of all elements in b

**Expected Output:**
```
Addition:
tensor([[ 6.,  8.],
        [10., 12.]])

Multiplication:
tensor([[ 5., 12.],
        [21., 32.]])

Matrix Multiplication:
tensor([[19., 22.],
        [43., 50.]])

Sum of a: tensor(10.)
Mean of b: tensor(6.5000)
```

**Sample Code:**
```python
# Tensor operations
x = torch.tensor([[1., 2.], [3., 4.]])
y = torch.tensor([[2., 0.], [1., 3.]])
added = x + y                    # Element-wise add
product = x * y                  # Element-wise multiply
matrix_prod = x @ y              # Matrix multiply
total = x.sum()                  # Sum all
print("Sum:", total)
```

In [ ]:
# Your code here


## 1.3 Automatic Differentiation (Autograd)

PyTorch can automatically compute gradients - this is essential for training neural networks.

Key concepts:
- `requires_grad=True`: Tell PyTorch to track operations for gradient computation
- `.backward()`: Compute gradients
- `.grad`: Access the computed gradient

Example:
```python
x = torch.tensor([2.0], requires_grad=True)
y = x ** 2  # y = x^2
y.backward()  # Compute dy/dx
print(x.grad)  # Should be 4.0 (derivative of x^2 is 2x)
```

**Your Task:**
1. Create a tensor `x = torch.tensor([3.0], requires_grad=True)`
2. Compute `y = x ** 3` (y = x cubed)
3. Call `y.backward()` to compute the gradient
4. Print `x.grad` (should be 27.0, since derivative of x³ is 3x² = 3×9 = 27)

**Expected Output:**
```
x = tensor([3.], requires_grad=True)
y = x^3 = tensor([27.], grad_fn=<PowBackward0>)
Gradient (dy/dx) = tensor([27.])
```

In [ ]:
# Your code here


---
# Part 2: Linear Regression

Linear regression finds the best-fit line: `y = wx + b`

## 2.1 Generate Data

Run the cell below to create training data with a known relationship: `y = 3x + 1 + noise`

In [ ]:
# Generate linear data: y = 3x + 1 + noise
np.random.seed(42)
X_np = np.random.rand(100, 1) * 10  # 100 samples, values 0-10
y_np = 3 * X_np + 1 + np.random.randn(100, 1) * 2  # y = 3x + 1 + noise

# Convert to PyTorch tensors
X = torch.FloatTensor(X_np)
y = torch.FloatTensor(y_np)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Visualize
plt.scatter(X_np, y_np, alpha=0.6)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Training Data (y = 3x + 1 + noise)')
plt.show()

## 2.2 Define a Linear Model

Create a model using `nn.Module`:

```python
class ModelName(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, x):
        return self.linear(x)
```

**Your Task:** Create a `LinearRegression` class:
1. Define `__init__` with `nn.Linear(1, 1)` (1 input feature, 1 output)
2. Define `forward` that returns `self.linear(x)`
3. Create an instance: `model = LinearRegression()`
4. Print the model

**Expected Output:**
```
LinearRegression(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)
```

In [ ]:
# Your code here


## 2.3 Train the Model

The PyTorch training loop:

```python
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

for epoch in range(n_epochs):
    # Forward pass
    y_pred = model(X)
    loss = criterion(y_pred, y)
    
    # Backward pass
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()        # Compute gradients
    optimizer.step()       # Update weights
```

**Your Task:** Train your linear regression model:
1. Create `criterion = nn.MSELoss()`
2. Create `optimizer = optim.SGD(model.parameters(), lr=0.01)`
3. Train for 100 epochs using the loop pattern above
4. Print the loss every 20 epochs
5. After training, print the learned weight and bias:
   - `model.linear.weight.item()` (should be close to 3)
   - `model.linear.bias.item()` (should be close to 1)

**Expected Output:**
```
Epoch 0, Loss: XX.XXXX
Epoch 20, Loss: X.XXXX
Epoch 40, Loss: X.XXXX
Epoch 60, Loss: X.XXXX
Epoch 80, Loss: X.XXXX

Learned weight: ~3.0 (close to 3)
Learned bias: ~1.0 (close to 1)
```

In [ ]:
# Your code here


---
# Part 3: Logistic Regression

Logistic regression is used for binary classification. It uses the sigmoid function to output probabilities.

## 3.1 Generate Classification Data

Run the cell below to create a binary classification dataset.

In [ ]:
# Generate classification data
from sklearn.datasets import make_classification

X_class, y_class = make_classification(
    n_samples=200, n_features=2, n_redundant=0, 
    n_informative=2, n_clusters_per_class=1, random_state=42
)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)

# Convert to tensors
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train).reshape(-1, 1)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.FloatTensor(y_test).reshape(-1, 1)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Visualize
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='coolwarm', alpha=0.6)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Binary Classification Data')
plt.colorbar(label='Class')
plt.show()

## 3.2 Define Logistic Regression Model

Logistic regression applies sigmoid to a linear transformation:

```python
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
```

**Your Task:**
1. Create a `LogisticRegression` class with 2 input features (matching our data)
2. Use `torch.sigmoid()` in the forward method
3. Create an instance: `log_model = LogisticRegression(2)`

**Expected Output:**
```
LogisticRegression(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
```

In [ ]:
# Your code here


## 3.3 Train Logistic Regression

For binary classification, use Binary Cross Entropy loss:

```python
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=0.1)
```

**Your Task:**
1. Create BCELoss criterion and SGD optimizer (lr=0.1)
2. Train for 200 epochs
3. Print loss every 40 epochs
4. After training, evaluate on test set:
   - Get predictions: `y_pred = (log_model(X_test_t) >= 0.5).float()`
   - Calculate accuracy: `(y_pred == y_test_t).float().mean()`

**Expected Output:**
```
Epoch 0, Loss: 0.XXXX
Epoch 40, Loss: 0.XXXX
Epoch 80, Loss: 0.XXXX
Epoch 120, Loss: 0.XXXX
Epoch 160, Loss: 0.XXXX

Test Accuracy: ~0.95 (around 95%)
```

In [ ]:
# Your code here


---
# Part 4: Support Vector Machines

SVMs find the optimal hyperplane that separates classes with maximum margin.

## 4.1 SVM with Different Kernels

scikit-learn's SVC (Support Vector Classifier):

```python
from sklearn.svm import SVC

svm = SVC(kernel='linear')  # or 'rbf', 'poly'
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
```

Available kernels:
- `'linear'`: Linear decision boundary
- `'rbf'`: Radial Basis Function (good for non-linear data)
- `'poly'`: Polynomial kernel

**Your Task:**
1. Train an SVM with `kernel='linear'` on X_train, y_train
2. Predict on X_test
3. Calculate and print the accuracy

**Expected Output:**
```
Linear SVM Accuracy: ~0.95 (around 95%)
```

In [ ]:
# Your code here


## 4.2 Compare Kernels on Non-Linear Data

Run the cell below to create moon-shaped data that's not linearly separable.

In [ ]:
# Create non-linear (moon-shaped) data
X_moons, y_moons = make_moons(n_samples=300, noise=0.2, random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_moons, y_moons, test_size=0.2, random_state=42
)

plt.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='coolwarm', alpha=0.6)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Moon-shaped Data (Non-linear)')
plt.show()

## 4.3 Train and Compare Kernels

**Your Task:** Train SVMs with different kernels and compare accuracies:
1. Train an SVM with `kernel='linear'`
2. Train an SVM with `kernel='rbf'`
3. Train an SVM with `kernel='poly'`
4. Print the accuracy for each kernel

Which kernel works best for this non-linear data?

**Expected Output:**
```
Linear kernel accuracy: ~0.88
RBF kernel accuracy: ~1.00 (best for non-linear data)
Poly kernel accuracy: ~0.93
```

**Sample Code:**
```python
# Training SVMs with different kernels
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_m, y_train_m)
pred_linear = svm_linear.predict(X_test_m)
acc_linear = accuracy_score(y_test_m, pred_linear)
print(f"Linear accuracy: {acc_linear:.2f}")
```

In [ ]:
# Your code here


---
# Part 5: Model Evaluation

Beyond accuracy: precision, recall, F1-score, and confusion matrices.

## 5.1 Confusion Matrix

A confusion matrix shows the breakdown of predictions:

```python
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)
# [[TN, FP],
#  [FN, TP]]
```

Where:
- TN: True Negatives (correctly predicted 0)
- FP: False Positives (incorrectly predicted 1)
- FN: False Negatives (incorrectly predicted 0)
- TP: True Positives (correctly predicted 1)

**Your Task:**
1. Use your best SVM model from Part 4 to get predictions on the moon test data
2. Create a confusion matrix
3. Print the confusion matrix

**Expected Output:**
```
Confusion Matrix:
[[29  0]
 [ 0 31]]
```

In [ ]:
# Your code here


## 5.2 Classification Report

Get precision, recall, and F1-score in one report:

```python
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred)
print(report)
```

Metrics:
- **Precision**: Of predicted positives, how many are actually positive?
- **Recall**: Of actual positives, how many did we predict correctly?
- **F1-score**: Harmonic mean of precision and recall

**Your Task:**
1. Generate a classification report for your SVM predictions
2. Print the report

**Expected Output:**
```
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      1.00      1.00        31

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60
```

In [ ]:
# Your code here


---
# Lab Complete!

## Summary

You learned:
- **PyTorch Tensors**: Create, manipulate, and use autograd
- **Linear Regression**: nn.Module, training loop, MSE loss
- **Logistic Regression**: Sigmoid, BCE loss, classification
- **SVMs**: Different kernels for linear/non-linear data
- **Evaluation**: Confusion matrix, precision, recall, F1-score